In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Tavares2017_prep.csv')

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [4]:
tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_train = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1']])

obs_test = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_test = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1']])

print(X_train.shape, X_test.shape)

(30464, 4) (30464, 4)


In [5]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [6]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [7]:
layers_neuron = [10, 10, 8, 5]
input_shape = 4
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 87        
                                                                 
Total params: 380 (1.48 KB)
Trainable params: 380 (1.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [9]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=4000)

Epoch 1/4000
1/1 [==============================] - 1s 567ms/step - loss: 164261.9531
Epoch 2/4000
1/1 [==============================] - 0s 6ms/step - loss: 163527.3750
Epoch 3/4000
1/1 [==============================] - 0s 5ms/step - loss: 163007.8125
Epoch 4/4000
1/1 [==============================] - 0s 6ms/step - loss: 162576.3438
Epoch 5/4000
1/1 [==============================] - 0s 6ms/step - loss: 162193.5156
Epoch 6/4000
1/1 [==============================] - 0s 6ms/step - loss: 161842.3750
Epoch 7/4000
1/1 [==============================] - 0s 6ms/step - loss: 161514.1250
Epoch 8/4000
1/1 [==============================] - 0s 6ms/step - loss: 161203.4062
Epoch 9/4000
1/1 [==============================] - 0s 5ms/step - loss: 160906.7656
Epoch 10/4000
1/1 [==============================] - 0s 6ms/step - loss: 160621.7500
Epoch 11/4000
1/1 [==============================] - 0s 6ms/step - loss: 160346.5000
Epoch 12/4000
1/1 [==============================] - 0s 5ms/step - loss:

1/1 [==============================] - 0s 9ms/step - loss: 144395.0625
Epoch 98/4000
1/1 [==============================] - 0s 9ms/step - loss: 144255.3750
Epoch 99/4000
1/1 [==============================] - 0s 9ms/step - loss: 144116.5781
Epoch 100/4000
1/1 [==============================] - 0s 9ms/step - loss: 143978.6875
Epoch 101/4000
1/1 [==============================] - 0s 8ms/step - loss: 143841.6562
Epoch 102/4000
1/1 [==============================] - 0s 9ms/step - loss: 143705.5312
Epoch 103/4000
1/1 [==============================] - 0s 9ms/step - loss: 143570.2344
Epoch 104/4000
1/1 [==============================] - 0s 9ms/step - loss: 143435.8438
Epoch 105/4000
1/1 [==============================] - 0s 10ms/step - loss: 143302.2969
Epoch 106/4000
1/1 [==============================] - 0s 9ms/step - loss: 143169.5938
Epoch 107/4000
1/1 [==============================] - 0s 9ms/step - loss: 143037.7344
Epoch 108/4000
1/1 [==============================] - 0s 9ms/step - lo

1/1 [==============================] - 0s 9ms/step - loss: 134018.5000
Epoch 193/4000
1/1 [==============================] - 0s 8ms/step - loss: 133925.7031
Epoch 194/4000
1/1 [==============================] - 0s 8ms/step - loss: 133833.0938
Epoch 195/4000
1/1 [==============================] - 0s 8ms/step - loss: 133740.6250
Epoch 196/4000
1/1 [==============================] - 0s 8ms/step - loss: 133648.3594
Epoch 197/4000
1/1 [==============================] - 0s 8ms/step - loss: 133556.2812
Epoch 198/4000
1/1 [==============================] - 0s 8ms/step - loss: 133464.4219
Epoch 199/4000
1/1 [==============================] - 0s 8ms/step - loss: 133372.7812
Epoch 200/4000
1/1 [==============================] - 0s 8ms/step - loss: 133281.3750
Epoch 201/4000
1/1 [==============================] - 0s 8ms/step - loss: 133190.2188
Epoch 202/4000
1/1 [==============================] - 0s 8ms/step - loss: 133099.3438
Epoch 203/4000
1/1 [==============================] - 0s 8ms/step - l

1/1 [==============================] - 0s 8ms/step - loss: 125308.2969
Epoch 288/4000
1/1 [==============================] - 0s 7ms/step - loss: 125204.7656
Epoch 289/4000
1/1 [==============================] - 0s 8ms/step - loss: 125101.2500
Epoch 290/4000
1/1 [==============================] - 0s 8ms/step - loss: 124997.7500
Epoch 291/4000
1/1 [==============================] - 0s 8ms/step - loss: 124894.2969
Epoch 292/4000
1/1 [==============================] - 0s 8ms/step - loss: 124790.8750
Epoch 293/4000
1/1 [==============================] - 0s 8ms/step - loss: 124687.4922
Epoch 294/4000
1/1 [==============================] - 0s 8ms/step - loss: 124584.1797
Epoch 295/4000
1/1 [==============================] - 0s 8ms/step - loss: 124480.9219
Epoch 296/4000
1/1 [==============================] - 0s 8ms/step - loss: 124377.7266
Epoch 297/4000
1/1 [==============================] - 0s 8ms/step - loss: 124274.5938
Epoch 298/4000
1/1 [==============================] - 0s 7ms/step - l

1/1 [==============================] - 0s 8ms/step - loss: 115856.8203
Epoch 383/4000
1/1 [==============================] - 0s 8ms/step - loss: 115762.6641
Epoch 384/4000
1/1 [==============================] - 0s 8ms/step - loss: 115668.5156
Epoch 385/4000
1/1 [==============================] - 0s 8ms/step - loss: 115574.6562
Epoch 386/4000
1/1 [==============================] - 0s 8ms/step - loss: 115480.8125
Epoch 387/4000
1/1 [==============================] - 0s 7ms/step - loss: 115387.2656
Epoch 388/4000
1/1 [==============================] - 0s 8ms/step - loss: 115293.7266
Epoch 389/4000
1/1 [==============================] - 0s 7ms/step - loss: 115200.4844
Epoch 390/4000
1/1 [==============================] - 0s 8ms/step - loss: 115107.2344
Epoch 391/4000
1/1 [==============================] - 0s 8ms/step - loss: 115014.3047
Epoch 392/4000
1/1 [==============================] - 0s 8ms/step - loss: 114921.3672
Epoch 393/4000
1/1 [==============================] - 0s 8ms/step - l

...

1/1 [==============================] - 0s 11ms/step - loss: 73447.5078
Epoch 3595/4000
1/1 [==============================] - 0s 10ms/step - loss: 73497.6328
Epoch 3596/4000
1/1 [==============================] - 0s 12ms/step - loss: 73497.9766
Epoch 3597/4000
1/1 [==============================] - 0s 10ms/step - loss: 73500.7500
Epoch 3598/4000
1/1 [==============================] - 0s 10ms/step - loss: 73458.0469
Epoch 3599/4000
1/1 [==============================] - 0s 10ms/step - loss: 73461.0078
Epoch 3600/4000
1/1 [==============================] - 0s 11ms/step - loss: 73513.2031
Epoch 3601/4000
1/1 [==============================] - 0s 10ms/step - loss: 73465.6016
Epoch 3602/4000
1/1 [==============================] - 0s 12ms/step - loss: 73457.7812
Epoch 3603/4000
1/1 [==============================] - 0s 11ms/step - loss: 73449.3516
Epoch 3604/4000
1/1 [==============================] - 0s 11ms/step - loss: 73497.4531
Epoch 3605/4000
1/1 [==============================] - 0s 1

1/1 [==============================] - 0s 10ms/step - loss: 73473.9531
Epoch 3689/4000
1/1 [==============================] - 0s 11ms/step - loss: 73434.4375
Epoch 3690/4000
1/1 [==============================] - 0s 11ms/step - loss: 73476.7969
Epoch 3691/4000
1/1 [==============================] - 0s 10ms/step - loss: 73435.7109
Epoch 3692/4000
1/1 [==============================] - 0s 11ms/step - loss: 73476.2422
Epoch 3693/4000
1/1 [==============================] - 0s 12ms/step - loss: 73432.9062
Epoch 3694/4000
1/1 [==============================] - 0s 10ms/step - loss: 73435.2344
Epoch 3695/4000
1/1 [==============================] - 0s 11ms/step - loss: 73438.4844
Epoch 3696/4000
1/1 [==============================] - 0s 10ms/step - loss: 73441.6172
Epoch 3697/4000
1/1 [==============================] - 0s 10ms/step - loss: 73449.5703
Epoch 3698/4000
1/1 [==============================] - 0s 11ms/step - loss: 73441.4219
Epoch 3699/4000
1/1 [==============================] - 0s 1

1/1 [==============================] - 0s 12ms/step - loss: 73473.7344
Epoch 3783/4000
1/1 [==============================] - 0s 14ms/step - loss: 73432.5391
Epoch 3784/4000
1/1 [==============================] - 0s 13ms/step - loss: 73484.3828
Epoch 3785/4000
1/1 [==============================] - 0s 12ms/step - loss: 73454.2578
Epoch 3786/4000
1/1 [==============================] - 0s 13ms/step - loss: 73430.2734
Epoch 3787/4000
1/1 [==============================] - 0s 11ms/step - loss: 73414.9531
Epoch 3788/4000
1/1 [==============================] - 0s 11ms/step - loss: 73450.8594
Epoch 3789/4000
1/1 [==============================] - 0s 12ms/step - loss: 73406.9062
Epoch 3790/4000
1/1 [==============================] - 0s 13ms/step - loss: 73408.3828
Epoch 3791/4000
1/1 [==============================] - 0s 12ms/step - loss: 73412.7578
Epoch 3792/4000
1/1 [==============================] - 0s 12ms/step - loss: 73414.1406
Epoch 3793/4000
1/1 [==============================] - 0s 1

1/1 [==============================] - 0s 13ms/step - loss: 73395.2734
Epoch 3877/4000
1/1 [==============================] - 0s 12ms/step - loss: 73399.6406
Epoch 3878/4000
1/1 [==============================] - 0s 11ms/step - loss: 73444.1016
Epoch 3879/4000
1/1 [==============================] - 0s 11ms/step - loss: 73408.1875
Epoch 3880/4000
1/1 [==============================] - 0s 10ms/step - loss: 73412.9844
Epoch 3881/4000
1/1 [==============================] - 0s 11ms/step - loss: 73419.9688
Epoch 3882/4000
1/1 [==============================] - 0s 12ms/step - loss: 73475.8750
Epoch 3883/4000
1/1 [==============================] - 0s 11ms/step - loss: 73433.1875
Epoch 3884/4000
1/1 [==============================] - 0s 11ms/step - loss: 73460.5781
Epoch 3885/4000
1/1 [==============================] - 0s 9ms/step - loss: 73408.1250
Epoch 3886/4000
1/1 [==============================] - 0s 11ms/step - loss: 73416.1172
Epoch 3887/4000
1/1 [==============================] - 0s 10

1/1 [==============================] - 0s 11ms/step - loss: 73443.6016
Epoch 3971/4000
1/1 [==============================] - 0s 11ms/step - loss: 73391.3750
Epoch 3972/4000
1/1 [==============================] - 0s 9ms/step - loss: 73385.0703
Epoch 3973/4000
1/1 [==============================] - 0s 12ms/step - loss: 73385.6875
Epoch 3974/4000
1/1 [==============================] - 0s 10ms/step - loss: 73383.7734
Epoch 3975/4000
1/1 [==============================] - 0s 9ms/step - loss: 73386.9844
Epoch 3976/4000
1/1 [==============================] - 0s 12ms/step - loss: 73390.4531
Epoch 3977/4000
1/1 [==============================] - 0s 10ms/step - loss: 73395.9062
Epoch 3978/4000
1/1 [==============================] - 0s 10ms/step - loss: 73409.5859
Epoch 3979/4000
1/1 [==============================] - 0s 10ms/step - loss: 73417.7266
Epoch 3980/4000
1/1 [==============================] - 0s 9ms/step - loss: 73423.8125
Epoch 3981/4000
1/1 [==============================] - 0s 10ms

In [10]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input.npy', X_test)
np.save('output.npy', pred_v)

952/952 [==============================] - 0s 263us/step


In [11]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df.item_value_1[s] - ts_df.item_value_0[s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv.to_csv('Tv2017_complete.csv')